In [11]:
!pip install gensim scikit-learn rouge-score nltk

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt', quiet=True)  # Tokenization
import warnings
warnings.filterwarnings('ignore')  # Suppress Gensim warnings
import os
import csv

def load_text(file_path):
    """Load text from file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except FileNotFoundError:
        print(f"Error: File {file_path} not found. Create it with sample text.")
        return ""

def get_word2vec_embedding(text, model, unknown_vector=np.zeros(300)):
    """Compute averaged Word2Vec embedding for a text."""
    tokens = word_tokenize(text.lower())
    valid_embeddings = []
    for token in tokens:
        if token in model:
            valid_embeddings.append(model[token])
        else:
            valid_embeddings.append(unknown_vector)  # Fallback for OOV
    if valid_embeddings:
        return np.mean(valid_embeddings, axis=0)
    return unknown_vector

def semantic_similarity(human_text, generated_text):
    if word2vec_model is None:
        return None  # Model unavailable → no similarity
    human_emb = get_word2vec_embedding(human_text, word2vec_model)
    gen_emb = get_word2vec_embedding(generated_text, word2vec_model)
    
    sim_matrix = cosine_similarity(human_emb.reshape(1, -1), gen_emb.reshape(1, -1))
    sim = sim_matrix[0][0]
    return sim

In [17]:
human_folder = 'testset/human_feedback'
model_folder = 'testset/model_feedback'

print("Loading Word2Vec model...")
word2vec_model = api.load('word2vec-google-news-300')
print("Model loaded!")

Loading Word2Vec model...
Model loaded!


In [ ]:
human_file_names = []
model_file_names = []
semantic_scores = []

print("\nWord2Vec Cosine Similarity Evaluation:\n")

for human_file in os.listdir(human_folder):

    human_path = os.path.join(human_folder, human_file)
    model_path = os.path.join(model_folder, human_file)

    if os.path.isfile(human_path) and os.path.isfile(model_path):

        human_text = load_text(human_path)
        generated_text = load_text(model_path)

        sem_sim = semantic_similarity(human_text, generated_text)

        human_file_names.append(human_file)
        model_file_names.append(human_file)
        semantic_scores.append(sem_sim)

        print(f"File Pair:")
        print(f"  Human: {human_file}")
        print(f"  Model: {human_file}")
        print(f"Semantic Similarity: {sem_sim}")

# Compute stats only for valid scores
valid_scores = [s for s in semantic_scores if s is not None]
semantic_array = np.array(valid_scores) if valid_scores else np.array([0])

mean_similarity = float(np.mean(semantic_array))
std_similarity = float(np.std(semantic_array))

print("\nStatistics")
print("Mean Similarity:", mean_similarity)
print("Standard Deviation:", std_similarity)

# Save to CSV
csv_file = 'semantic_similarity_results.csv'

with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)

    writer.writerow(["Human Feedback File", "Model Feedback File", "Semantic Similarity"])

    for hf, mf, sim in zip(human_file_names, model_file_names, semantic_scores):
        writer.writerow([hf, mf, sim])

    writer.writerow([])
    writer.writerow(["Mean Similarity", mean_similarity])
    writer.writerow(["Standard Deviation", std_similarity])

print("\nResults saved to semantic_similarity_results.csv")


Word2Vec Cosine Similarity Evaluation:

File Pair:
  Human: 2e8FnaI8skE_5_0.txt
  Model: 2e8FnaI8skE_5_0.txt
Semantic Similarity: 0.8374258380065621
File Pair:
  Human: 2RaNq8rmsGA_8_0.txt
  Model: 2RaNq8rmsGA_8_0.txt
Semantic Similarity: 0.8849683811262997

Statistics
Mean Similarity: 0.8611971095664309
Standard Deviation: 0.0237712715598688

Results saved to semantic_similarity_results.csv
